<a href="https://colab.research.google.com/github/NekoDK/Omgu_DA/blob/OmguDA_1/OmguDA_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_excel('russiaxlsx.xlsx')

In [ ]:
df.head()

A = Число избирателей, внесенных в список избирателей на момент окончания голосования

B = Число избирательных бюллетеней, полученных участковой избирательной комиссией

C = Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно

D = Число избирательных бюллетеней, выданных в помещении для голосования в день голосования

E = пустой

F = Число погашенных избирательных бюллетеней

G = Число избирательных бюллетеней, содержащихся в переносных ящиках для голосования

H = Число избирательных бюллетеней, содержащихся в стационарных ящиках для голосования

I = Число недействительных избирательных бюллетеней

J = Число действительных избирательных бюллетеней

K = Число утраченных избирательных бюллетеней

L = Число избирательных бюллетеней, не учтенных при получении

In [ ]:
print("_______Gen_Old_______")
print(df['A'].std())
print(df['A'].mean())

_______Gen_Old_______
1914.2176931446295
1133.7104801453413


In [ ]:
step = 200
start = 94
genErr = 3 * df['A'].std()

df = df[df['A'] < genErr]

rd = df.sample(500, replace=False)

sf = df.iloc[start::step]

In [ ]:
def assign_value(row):
    if row['A'] >= 500000:
        return 1
    elif 100000 <= row['A'] < 500000:
        return 2
    elif 50000 <= row['A'] < 100000:
        return 3
    elif 25000 <= row['A'] < 50000:
        return 4
    elif 5000 <= row['A'] < 25000:
        return 5
    elif 1000 <= row['A'] < 5000:
        return 6
    elif 600 <= row['A'] < 1000:
        return 7
    elif row['A'] < 600:
        return 8
    else:
        return None

In [ ]:
df_city = df[df['tik'].str.contains(',')].copy()
df_city.loc['tik'] = df_city['tik'].str.split(',').str[0]
df_city.loc["tik"] = df_city["reg"]+" " +df_city["tik"]
df_city_big = df_city.copy()
df_city = df_city.groupby('tik').sum().reset_index()
df_country = df[~df['tik'].str.contains(',')].copy()
df_country.loc["tik"] = df_country["reg"]+" " + df_country["tik"]

df_city['our_group'] = df_city.apply(assign_value, axis=1)
df_country['our_group'] = df_country.apply(assign_value, axis=1)
df_city_big = df_city_big.merge(df_city[['tik', 'our_group']], on='tik', how='left')
df_city_big[df_city_big['tik'] == 'Московская область Москва']
df_all = pd.concat([df_city_big,df_country],axis = 0)

In [ ]:
df_new = df_all[df_all['our_group'] == 1]
sample_size = int(0.20 * len(df_new))
df_new = df_new.sample(n=sample_size)

In [ ]:
for i in range(2, 9):
  filtered_df = df_all[df_all['our_group'] == i]
  sample_size = int(0.20 * len(filtered_df))
  sampled_df = filtered_df.sample(n=sample_size)
  df_new = pd.concat([df_new,sampled_df],axis = 0)

In [ ]:
print("_______Gen_______")
print(df['A'].std())
print(df['A'].mean())
print("-----Rand----------")
print(rd['A'].std())
print(rd['A'].mean())
print("-----Step----------")
print(sf['A'].std())
print(sf['A'].mean())
print(("----Typezed-------"))
print(df_new['A'].std())
print(df_new['A'].mean())

_______Gen_______
794.1688866206755
1106.441364627687
-----Rand----------
790.4816250229972
1144.33
-----Step----------
772.0859655964242
1132.9417879417879
----Typezed-------
796.2670597498085
1106.9741898629
